In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers, losses, regularizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def build_efficientnetb0_model(input_shape, num_classes):
    # Load the pre-trained EfficientNetB0 model
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)
    
    # Add custom head for classification with L1 and L2 regularization
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(num_classes, 
                     activation='softmax', 
                     kernel_regularizer=regularizers.l2(0.01),  # L2 regularization
                     activity_regularizer=regularizers.l1(0.01))(x)  # L1 regularization
    
    # Create the model
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    
    return model



# Define parameters and paths
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
NUM_CLASSES = 5
EPOCHS = 50
BATCH_SIZE = 32

TRAIN_PATH = "/kaggle/input/heart-stages/HEART_TTV/train"
VAL_PATH = "/kaggle/input/heart-stages/HEART_TTV/validation"
TEST_PATH = "/kaggle/input/heart-stages/HEART_TTV/validation"


training_file = "D:\tra_3.xlsx"
validation_file = "D:\val_3.xlsx"
test_file = "D:\test_3.xlsx"



# Data generators
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Flow training images in batches from directory
train_ds = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow validation images in batches from directory
val_ds = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow test images in batches from directory
test_ds = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)  # Input shape for RGB images

# Build the model
model = build_efficientnetb0_model(input_shape, NUM_CLASSES)
optimizer = optimizers.RMSprop(learning_rate=1e-4)  # Using RMSprop optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

# Evaluate on validation dataset
val_loss, val_accuracy = model.evaluate(val_ds)

# Evaluate on test dataset
test_loss, test_accuracy = model.evaluate(test_ds)

# Save metrics to Excel files
#save_metrics_to_excel(history, val_loss, val_accuracy, test_loss, test_accuracy, training_file, validation_file, test_file)

# Print final metrics
print("Training Metrics:")
print("Train Loss:", history.history['loss'][-1])
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Metrics:")
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)
print("Test Metrics:")
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
from sklearn.metrics import classification_report
import numpy as np
predictions = model.predict(test_ds)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_labels = test_ds.classes

# Compute classification report
class_names = list(test_ds.class_indices.keys())
report = classification_report(true_labels, predicted_classes, target_names=class_names)

# Print classification report
print("\nClassification Report:")
print(report)

In [ ]:
from sklearn.metrics import classification_report
def calculate_classification_report(model, dataset):
    # Predict classes
    predictions = model.predict(dataset)
    y_pred = tf.argmax(predictions, axis=1).numpy()  # Convert predicted probabilities to class labels
    y_true = dataset.classes

    # Get class labels
    class_labels = list(dataset.class_indices.keys())

    # Generate classification report
    report = classification_report(y_true, y_pred, target_names=class_labels, output_dict=True)
    return report

train_report = calculate_classification_report(model, train_ds)
val_report = calculate_classification_report(model, val_ds)
test_report = calculate_classification_report(model, test_ds)

# Save metrics to Excel files (if needed)
save_metrics_to_excel(history, val_loss, val_accuracy, test_loss, test_accuracy, training_file, validation_file, test_file)

# Print final metrics
print("Training Metrics:")
print("Train Loss:", history.history['loss'][-1])
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Metrics:")
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)
print("Test Metrics:")
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print classification reports
print("\nClassification Report - Train:")
print(pd.DataFrame(train_report).transpose())
print("\nClassification Report - Validation:")
print(pd.DataFrame(val_report).transpose())
print("\nClassification Report - Test:")
print(pd.DataFrame(test_report).transpose())

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(10, 6))

# Plot training accuracy in red
plt.plot(history.history['accuracy'], label='Train Accuracy', color='red', linewidth=2.5)

# Plot validation accuracy in default blue
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2.5)

# Customize plot appearance
plt.title('Model Accuracy', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy', fontsize=12, fontweight='bold')
plt.xticks(fontsize=10, fontweight='bold')  # Set font weight of x-axis tick labels to bold
plt.yticks(fontsize=10, fontweight='bold')  # Set font weight of y-axis tick labels to bold
plt.grid(False)

# Add legend
plt.legend(loc='lower right', fontsize=12)

# Show plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.figure(figsize=(10, 6))

# Plot training loss in red
plt.plot(history.history['loss'], label='Train Loss', color='red', linewidth=2.5)

# Plot validation loss in default blue
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2.5)

# Customize plot appearance
plt.title('Model Loss', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12, fontweight='bold')
plt.ylabel('Loss', fontsize=12, fontweight='bold')
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(False)

# Set font weight of x-axis and y-axis labels to bold
plt.xlabel('Epoch', fontsize=12, fontweight='bold')
plt.ylabel('Loss', fontsize=12, fontweight='bold')

# Add legend
plt.legend(loc='upper right', fontsize=12)

# Show plot
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors

def plot_confusion_matrix(model, data_generator, dataset_name):
    # Get true labels from data generator
    y_true = data_generator.classes
    
    # Predict labels
    y_pred_prob = model.predict(data_generator)
    y_pred = np.argmax(y_pred_prob, axis=1)
    
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Get class labels
    class_labels = list(data_generator.class_indices.keys())
    
    # Define custom colormap with softer shades of green
    cmap = mcolors.ListedColormap(['#C3E6CB', '#A2D9A3', '#7FBF7B', '#5DA55D', '#3C8B3C'])
    
    # Plot confusion matrix with custom colormap
    plt.figure(figsize=(8, 8))
    
    # Plot confusion matrix with black lines between cells
    plt.imshow(cm, interpolation='nearest', cmap=cmap, extent=[-0.5, len(class_labels) - 0.5, -0.5, len(class_labels) - 0.5], origin='upper')
    
    # Add title
    plt.title(f'Confusion Matrix - {dataset_name}', fontsize=14, fontweight='bold')
    
    # Add class labels with bold font
    tick_marks = np.arange(len(class_labels))
    plt.xticks(tick_marks, class_labels, rotation=45, fontsize=10, ha='right', rotation_mode='anchor', fontweight='bold')
    plt.yticks(tick_marks, class_labels, fontsize=10, fontweight='bold')
    
    # Loop through data and add text annotations (bold)
    thresh = cm.max() / 2.0
    for i in range(len(class_labels)):
        for j in range(len(class_labels)):
            plt.text(j, i, f'{cm[i, j]}', ha='center', va='center', color='black' if cm[i, j] > thresh else 'black', fontweight='bold')
    
    # Set axis labels
    plt.xlabel('Predicted Label', fontsize=12, fontweight='bold')
    plt.ylabel('True Label', fontsize=12, fontweight='bold')
    
    # Add gridlines between cells
    for i in range(len(class_labels)):
        plt.axhline(i - 0.5, color='black', linewidth=1)
        plt.axvline(i - 0.5, color='black', linewidth=1)
    
    # Hide axis ticks
    plt.tick_params(axis='both', which='both', length=0)
    
    # Show plot
    plt.show()

# Example usage:

# Plot confusion matrix for train dataset
plot_confusion_matrix(model, train_ds, 'Train')

# Plot confusion matrix for validation dataset
plot_confusion_matrix(model, val_ds, 'Validation')

# Plot confusion matrix for test dataset
plot_confusion_matrix(model, test_ds, 'Test')


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors

def plot_confusion_matrix(model, data_generator, dataset_name):
    # Get true labels from data generator
    y_true = data_generator.classes
    
    # Predict labels
    y_pred_prob = model.predict(data_generator)
    y_pred = np.argmax(y_pred_prob, axis=1)
    
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Get class labels
    class_labels = list(data_generator.class_indices.keys())
    
    # Define custom colormap with softer shades of green
    cmap = mcolors.ListedColormap(['#C3E6CB', '#A2D9A3', '#7FBF7B', '#5DA55D', '#3C8B3C'])
    
    # Plot confusion matrix with custom colormap
    plt.figure(figsize=(8, 8))
    
    # Plot confusion matrix with black lines between cells
    plt.imshow(cm, interpolation='nearest', cmap=cmap, extent=[-0.5, len(class_labels) - 0.5, -0.5, len(class_labels) - 0.5], origin='upper')
    
    # Add title
    plt.title(f'Confusion Matrix - {dataset_name}', fontsize=14, fontweight='bold')

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers, regularizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def build_efficientnetb0_model(input_shape, num_classes):
    # Load the pre-trained EfficientNetB0 model
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)
    
    # Add custom head for classification with L1 and L2 regularization
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(num_classes, 
                     activation='softmax', 
                     kernel_regularizer=regularizers.l2(0.01),  # L2 regularization
                     activity_regularizer=regularizers.l1(0.01))(x)  # L1 regularization
    
    # Create the model
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    
    return model

In [ ]:
def save_metrics_to_excel(history, val_loss, val_accuracy, test_loss, test_accuracy, training_file, validation_file, test_file):
    # Extract training and validation metrics from history object
    training_loss = history.history['loss']
    training_accuracy = history.history['accuracy']
    validation_loss = history.history['val_loss']
    validation_accuracy = history.history['val_accuracy']

    # Create DataFrames for training, validation, and test metrics
    training_metrics_df = pd.DataFrame({
        'Training Loss': training_loss,
        'Training Accuracy': training_accuracy
    })
    validation_metrics_df = pd.DataFrame({
        'Validation Loss': validation_loss,
        'Validation Accuracy': validation_accuracy
    })
    test_metrics_df = pd.DataFrame({
        'Test Loss': [test_loss],
        'Test Accuracy': [test_accuracy]
    })

    # Save metrics to specified Excel files
    training_metrics_df.to_excel(training_file, index=False)
    validation_metrics_df.to_excel(validation_file, index=False)
    test_metrics_df.to_excel(test_file, index=False)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def build_unet_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    
    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bottleneck
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)

    # Decoder
    up4 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv3)
    up4 = layers.concatenate([up4, conv2], axis=3)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(up4)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up5 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv4)
    up5 = layers.concatenate([up5, conv1], axis=3)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(up5)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv5)

    outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(conv5)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    return model

# Define parameters
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
NUM_CLASSES = 5
EPOCHS = 50
BATCH_SIZE = 32

TRAIN_PATH = "/path/to/train"  # Update with your training data directory
VAL_PATH = "/path/to/validation"  # Update with your validation data directory
TEST_PATH = "/path/to/test"  # Update with your test data directory

# Data generators
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Flow training images in batches from directory
train_ds = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='input',  # Set to 'input' for semantic segmentation
)

# Flow validation images in batches from directory
val_ds = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='input',  # Set to 'input' for semantic segmentation
)

# Flow test images in batches from directory
test_ds = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='input',  # Set to 'input' for semantic segmentation
)

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)  # Input shape for RGB images

# Build the U-Net model
# Compile the model with sparse categorical cross-entropy loss
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

# Evaluate on validation dataset
val_loss, val_accuracy = model.evaluate(val_ds)


# Train the model
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

# Evaluate on validation dataset
val_loss, val_accuracy = model.evaluate(val_ds)

# Evaluate on test dataset
test_loss, test_accuracy = model.evaluate(test_ds)

# Print final metrics
print("Training Metrics:")
print("Train Loss:", history.history['loss'][-1])
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Metrics:")
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)
print("Test Metrics:")
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, regularizers, callbacks
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def build_densenet_model(input_shape, num_classes):
    # Load the pre-trained DenseNet169 model
    base_model = DenseNet169(include_top=False, weights='imagenet', input_shape=input_shape)
    
    # Set the base model layers to be trainable (no freezing)
    base_model.trainable = True
    
    # Add custom head for classification with Dropout and L2 regularization
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dropout(0.5)(x)  # Add dropout layer
    x = layers.BatchNormalization()(x)  # Add batch normalization
    x = layers.Dense(num_classes, 
                     activation='softmax', 
                     kernel_regularizer=regularizers.l2(0.001))(x)  # L2 regularization
    
    # Create the model
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    
    return model

# Define parameters and paths
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
NUM_CLASSES = 5
EPOCHS = 50
BATCH_SIZE = 32

TRAIN_PATH = '/kaggle/input/heart-31/HEART_3_TTV/train' # Replace with your training data path
VAL_PATH = '/kaggle/input/heart-31/HEART_3_TTV/validation' # Replace with your validation data path
TEST_PATH = '/kaggle/input/heart-31/HEART_3_TTV/test' # Replace with your test data path

# Data generators
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    channel_shift_range=0.1
)

val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Flow training images in batches from directory
train_ds = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow validation images in batches from directory
val_ds = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow test images in batches from directory
test_ds = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)  # Input shape for RGB images

# Build the model
model = build_densenet_model(input_shape, NUM_CLASSES)

# Use Adam optimizer
optimizer = optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Add callbacks for early stopping and learning rate reduction
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)

# Train the model with learning rate scheduler and early stopping
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=[early_stopping, reduce_lr])

# Evaluate on validation dataset
val_loss, val_accuracy = model.evaluate(val_ds)

# Evaluate on test dataset
test_loss, test_accuracy = model.evaluate(test_ds)

# Print final metrics
print("Training Metrics:")
print("Train Loss:", history.history['loss'][-1])
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Metrics:")
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)
print("Test Metrics:")
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB7, preprocess_input
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image 
import keras
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
train_path= '/kaggle/input/heart-31/HEART_3_TTV/train' # Replace with your training data path
val_path = '/kaggle/input/heart-31/HEART_3_TTV/validation' # Replace with your validation data path
test_path = '/kaggle/input/heart-31/HEART_3_TTV/test' # Replace with your test data path
 
train_gen = ImageDataGenerator( horizontal_flip = True,
                                        rotation_range=30,
                                        width_shift_range=0.2,
                                        brightness_range=[0.2,1.2], 
                                       preprocessing_function=preprocess_input )
val_gen = ImageDataGenerator( preprocessing_function=preprocess_input )


train_generator = train_gen.flow_from_directory(
        directory= train_path ,
        target_size=(224, 224), 
        batch_size=32,
         shuffle=True,
        class_mode='categorical',
       )

validation_generator = val_gen.flow_from_directory(
         directory=val_path ,
        target_size=(224, 224),  
        batch_size=32,
        shuffle=False,
        class_mode='categorical')

print(validation_generator.class_indices)

In [ ]:
base_model = EfficientNetB7(input_shape = (224,224,3), include_top = False, weights = "imagenet",pooling= 'avg' )

for layers in base_model.layers:
    layers.trainable = False

In [ ]:
from tensorflow.keras import layers
from keras.models import Model

# add new classifier layers

CNN = base_model.output
#add Dense layer
CNN = layers.Dense(64, activation='relu')(CNN)
#add output layer with softmax activation
output = layers.Dense(5, activation='softmax')(CNN)
# define new model
EfficienNetB7_model = Model(inputs=base_model.inputs, outputs=output)
# summarize
#EfficienNetB7_model.summary()

In [ ]:
callbacks = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 2)
lr_rate = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=3, min_lr=0.0001)

EfficienNetB7_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-6),  
    loss=tf.keras.losses.CategoricalCrossentropy(
    from_logits=False, name='categorical_crossentropy'),
    metrics=[tf.keras.metrics.CategoricalAccuracy(
    name='accuracy')])

history = EfficienNetB7_model.fit(train_generator, epochs=50, validation_data=validation_generator, callbacks =[callbacks, lr_rate])

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

# Number of classes
num_classes = 5

# Load the ResNet50 model pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model with L2 regularization
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling layer
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)  # Dense layer with ReLU activation and L2 regularization
x = MaxPooling2D(pool_size=(2, 2))(x)  # Max Pooling layer
predictions = Dense(num_classes, activation='softmax', kernel_regularizer=l2(0.01))(x)  # Output layer with softmax activation and L2 regularization

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Define the directories for the training, validation, and test datasets
train_dir = '/kaggle/input/heart-stages/HEART_TTV/train'
val_dir = '/kaggle/input/heart-stages/HEART_TTV/validation'
test_dir = '/kaggle/input/heart-stages/HEART_TTV/test'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Custom callback to print metrics after each epoch
class PrintMetrics(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f'Epoch {epoch + 1}')
        print(f'Train Loss: {logs["loss"]:.4f}, Train Accuracy: {logs["accuracy"]:.4f}')
        print(f'Validation Loss: {logs["val_loss"]:.4f}, Validation Accuracy: {logs["val_accuracy"]:.4f}')
        print('-' * 30)

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[PrintMetrics()]
)

# Optionally, unfreeze some of the deeper layers and fine-tune the model
for layer in base_model.layers[-50:]:  # Unfreeze the last 50 layers for fine-tuning
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training with fine-tuning
model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[PrintMetrics()]
)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

# Number of classes
num_classes = 5

# Load the ResNet50 model pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model with L2 regularization
x = base_model.output
x = MaxPooling2D(pool_size=(2, 2))(x)  # Max Pooling layer
x = Flatten()(x)  # Flatten the output before the dense layers
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)  # Dense layer with ReLU activation and L2 regularization
predictions = Dense(num_classes, activation='softmax', kernel_regularizer=l2(0.01))(x)  # Output layer with softmax activation and L2 regularization

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Define the directories for the training, validation, and test datasets
train_dir = '/kaggle/input/heart-stages/HEART_TTV/train'
val_dir = '/kaggle/input/heart-stages/HEART_TTV/validation'
test_dir = '/kaggle/input/heart-stages/HEART_TTV/test'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Custom callback to print metrics after each epoch
class PrintMetrics(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f'Epoch {epoch + 1}')
        print(f'Train Loss: {logs["loss"]:.4f}, Train Accuracy: {logs["accuracy"]:.4f}')
        print(f'Validation Loss: {logs["val_loss"]:.4f}, Validation Accuracy: {logs["val_accuracy"]:.4f}')
        print('-' * 30)

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[PrintMetrics()]
)

# Optionally, unfreeze some of the deeper layers and fine-tune the model
for layer in base_model.layers[-50:]:  # Unfreeze the last 50 layers for fine-tuning
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training with fine-tuning
model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[PrintMetrics()]
)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

# Number of classes
num_classes = 5

# Load the ResNet50 model pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model with L2 regularization
x = base_model.output
x = MaxPooling2D(pool_size=(2, 2))(x)  # Max Pooling layer
x = Flatten()(x)  # Flatten the output before the dense layers
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)  # Dense layer with ReLU activation and L2 regularization
predictions = Dense(num_classes, activation='softmax', kernel_regularizer=l2(0.001))(x)  # Output layer with softmax activation and L2 regularization

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Define the directories for the training, validation, and test datasets
train_dir = '/kaggle/input/heart-stages/HEART_TTV/train'
val_dir = '/kaggle/input/heart-stages/HEART_TTV/validation'
test_dir = '/kaggle/input/heart-stages/HEART_TTV/test'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Custom callback to print metrics after each epoch
class PrintMetrics(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f'Epoch {epoch + 1}')
        print(f'Train Loss: {logs["loss"]:.4f}, Train Accuracy: {logs["accuracy"]:.4f}')
        print(f'Validation Loss: {logs["val_loss"]:.4f}, Validation Accuracy: {logs["val_accuracy"]:.4f}')
        print('-' * 30)

# Train the model
model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[PrintMetrics()]
)

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training with fine-tuning
model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[PrintMetrics()]
)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

# Number of classes
num_classes = 5

# Load the ResNet50 model pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model with L2 regularization
x = base_model.output
x = MaxPooling2D(pool_size=(2, 2))(x)  # Max Pooling layer
x = Flatten()(x)  # Flatten the output before the dense layers
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)  # Dense layer with ReLU activation and L2 regularization
predictions = Dense(num_classes, activation='softmax', kernel_regularizer=l2(0.001))(x)  # Output layer with softmax activation and L2 regularization

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Define the directories for the training, validation, and test datasets
train_dir = '/kaggle/input/heart-31/HEART_3_TTV/train'
val_dir = '/kaggle/input/heart-31/HEART_3_TTV/validation'
test_dir = '/kaggle/input/heart-31/HEART_3_TTV/test'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Custom callback to print metrics after each epoch
class PrintMetrics(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f'Epoch {epoch + 1}')
        print(f'Train Loss: {logs["loss"]:.4f}, Train Accuracy: {logs["accuracy"]:.4f}')
        print(f'Validation Loss: {logs["val_loss"]:.4f}, Validation Accuracy: {logs["val_accuracy"]:.4f}')
        print('-' * 30)

# Train the model
model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[PrintMetrics()]
)

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training with fine-tuning
model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[PrintMetrics()]
)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Define paths to your data directories
train_dir = '/kaggle/input/heart-31/HEART_3_TTV/train'
validation_dir = '/kaggle/input/heart-31/HEART_3_TTV/validation'
test_dir = '/kaggle/input/heart-31/HEART_3_TTV/test'

# === New Section for Evaluation and Prediction === #


In [ ]:
# Image dimensions and batch size
img_width, img_height = 224, 224  # ResNet50 default image size
batch_size = 32

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
# Load the ResNet50 model, pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Model architecture
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(5, activation='softmax')  # Output layer for 4 classes
])

# Freeze the layers of the base_model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model and save the history
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

# Save the model
model.save('lung_cancer_classification_model.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

# Number of classes
num_classes = 5

# Load the InceptionV3 model pre-trained on ImageNet
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model with L2 regularization
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling layer
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)  # Dense layer with ReLU activation and L2 regularization
x = MaxPooling2D(pool_size=(2, 2))(x)  # Max Pooling layer
predictions = Dense(num_classes, activation='softmax', kernel_regularizer=l2(0.01))(x)  # Output layer with softmax activation and L2 regularization

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Define the directories for the training, validation, and test datasets
train_dir = '/kaggle/input/heart-31/HEART_3_TTV/train'
val_dir = '/kaggle/input/heart-31/HEART_3_TTV/validation'
test_dir = '/kaggle/input/heart-31/HEART_3_TTV/test'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Custom callback to print metrics after each epoch
class PrintMetrics(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f'Epoch {epoch + 1}')
        print(f'Train Loss: {logs["loss"]:.4f}, Train Accuracy: {logs["accuracy"]:.4f}')
        print(f'Validation Loss: {logs["val_loss"]:.4f}, Validation Accuracy: {logs["val_accuracy"]:.4f}')
        print('-' * 30)

# Train the model
model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[PrintMetrics()]
)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers
from tensorflow.keras.applications import EfficientNetB0

def build_efficientnetb0_model(input_shape):
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)
    x = layers.GlobalAveragePooling2D()(base_model.output)
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    return model


In [ ]:
import cv2
import numpy as np
from skimage.feature import greycomatrix, greycoprops
from skimage.color import rgb2gray

def extract_glcm_features(image, distances=[5], angles=[0]):
    gray_image = rgb2gray(image)  # Use rgb2gray from skimage
    glcm = greycomatrix((gray_image * 255).astype(np.uint8), distances=distances, angles=angles, symmetric=True, normed=True)
    contrast = greycoprops(glcm, 'contrast').flatten()
    energy = greycoprops(glcm, 'energy').flatten()
    homogeneity = greycoprops(glcm, 'homogeneity').flatten()
    correlation = greycoprops(glcm, 'correlation').flatten()
    return np.hstack((contrast, energy, homogeneity, correlation))

def extract_shape_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return np.zeros(8)  # Return zero features if no contours are found
    cnt = contours[0]
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)
    moments = cv2.moments(cnt)
    hu_moments = cv2.HuMoments(moments).flatten()
    return np.hstack((area, perimeter, hu_moments))


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers
from tensorflow.keras.applications import EfficientNetB0

def build_efficientnetb0_model(input_shape):
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)
    x = layers.GlobalAveragePooling2D()(base_model.output)
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    return model


In [ ]:
########EFFICIENT NET B2 MODEL.......................................................

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers, losses, regularizers, callbacks
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def build_efficientnetb2_model(input_shape, num_classes):
    # Load the pre-trained EfficientNetB2 model
    base_model = EfficientNetB2(include_top=False, weights='imagenet', input_shape=input_shape)
    
    # Set the base model layers to be trainable (no freezing)
    base_model.trainable = True
    
    # Add custom head for classification with Dropout and L1, L2 regularization
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dropout(0.5)(x)  # Add dropout layer
    x = layers.BatchNormalization()(x)  # Add batch normalization
    x = layers.Dense(num_classes, 
                     activation='softmax', 
                     kernel_regularizer=regularizers.l2(0.01),  # L2 regularization
                     activity_regularizer=regularizers.l1(0.01))(x)  # L1 regularization
    
    # Create the model
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    
    return model

# Define parameters and paths
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
NUM_CLASSES = 5
EPOCHS = 50
BATCH_SIZE = 32

TRAIN_PATH = '/kaggle/input/heart-31/HEART_3_TTV/train' # Replace with your training data path
VAL_PATH = '/kaggle/input/heart-31/HEART_3_TTV/validation' # Replace with your validation data path
TEST_PATH = '/kaggle/input/heart-31/HEART_3_TTV/test' # Replace with your test data path

# Data generators
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    channel_shift_range=0.1
)

val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Flow training images in batches from directory
train_ds = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow validation images in batches from directory
val_ds = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow test images in batches from directory
test_ds = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)  # Input shape for RGB images

# Build the model
model = build_efficientnetb2_model(input_shape, NUM_CLASSES)

# Use Adam optimizer
optimizer = optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Add callbacks for early stopping and learning rate reduction
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)

# Train the model with learning rate scheduler and early stopping
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=[early_stopping, reduce_lr])

# Evaluate on validation dataset
val_loss, val_accuracy = model.evaluate(val_ds)

# Evaluate on test dataset
test_loss, test_accuracy = model.evaluate(test_ds)

# Save metrics to Excel files (you can define this function as needed)
# save_metrics_to_excel(history, val_loss, val_accuracy, test_loss, test_accuracy, training_file, validation_file, test_file)

# Print final metrics
print("Training Metrics:")
print("Train Loss:", history.history['loss'][-1])
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Metrics:")
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)
print("Test Metrics:")
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def generate_classification_report_and_confusion_matrix(model, dataset, dataset_type):
    # Predict classes
    predictions = model.predict(dataset)
    y_pred = np.argmax(predictions, axis=1)  # Convert predicted probabilities to class labels
    y_true = dataset.classes

    # Get class labels
    class_labels = list(dataset.class_indices.keys())

    # Generate classification report
    report = classification_report(y_true, y_pred, target_names=class_labels)
    print(f"\n{dataset_type} Classification Report:")
    print(report)

    # Generate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, class_labels, f'Confusion Matrix - {dataset_type}')

    return report

def plot_confusion_matrix(cm, class_labels, title):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=class_labels, yticklabels=class_labels, cbar=False,
                annot_kws={"weight": "bold"})
    plt.title(title, fontweight='bold')
    plt.xlabel('PREDICTED LABEL', fontweight='bold')
    plt.ylabel('TRUE LABEL', fontweight='bold')
    plt.xticks(fontweight='bold')
    plt.yticks(fontweight='bold')
    plt.show()

# Assuming `model`, `train_ds`, `val_ds`, and `test_ds` are already defined elsewhere in your code

# Generate classification reports and confusion matrices
train_report = generate_classification_report_and_confusion_matrix(model, train_ds, 'Training')
val_report = generate_classification_report_and_confusion_matrix(model, val_ds, 'Validation')
test_report = generate_classification_report_and_confusion_matrix(model, test_ds, 'Test')


In [ ]:
model.save('efficientnetB2_721_adam.h5')

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(10, 6))

# Plot training accuracy in red
plt.plot(history.history['accuracy'], label='Train Accuracy', color='red', linewidth=2.5)

# Plot validation accuracy in default blue
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2.5)

# Customize plot appearance
plt.title('Model Accuracy', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy', fontsize=12, fontweight='bold')
plt.xticks(fontsize=10, fontweight='bold')  # Set font weight of x-axis tick labels to bold
plt.yticks(fontsize=10, fontweight='bold')  # Set font weight of y-axis tick labels to bold
plt.grid(False)

# Add legend
plt.legend(loc='lower right', fontsize=12)

# Show plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.figure(figsize=(10, 6))

# Plot training loss in red
plt.plot(history.history['loss'], label='Train Loss', color='red', linewidth=2.5)

# Plot validation loss in default blue
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2.5)

# Customize plot appearance
plt.title('Model Loss', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12, fontweight='bold')
plt.ylabel('Loss', fontsize=12, fontweight='bold')
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(False)

# Set font weight of x-axis and y-axis labels to bold
plt.xlabel('Epoch', fontsize=12, fontweight='bold')
plt.ylabel('Loss', fontsize=12, fontweight='bold')

# Add legend
plt.legend(loc='upper right', fontsize=12)

# Show plot
plt.show()

In [ ]:
###########################################################################################################################################################################
###EFFICIENT NET B2 WITH ADAMW........................................
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers, regularizers, callbacks
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np

def build_efficientnetb2_model(input_shape, num_classes):
    # Load the pre-trained EfficientNetB2 model
    base_model = EfficientNetB2(include_top=False, weights='imagenet', input_shape=input_shape)
    
    # Set the base model layers to be trainable (no freezing)
    base_model.trainable = True
    
    # Add custom head for classification with Dropout and L1, L2 regularization
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dropout(0.5)(x)  # Add dropout layer
    x = layers.BatchNormalization()(x)  # Add batch normalization
    x = layers.Dense(num_classes, 
                     activation='softmax', 
                     kernel_regularizer=regularizers.l2(0.01),  # L2 regularization
                     activity_regularizer=regularizers.l1(0.01))(x)  # L1 regularization
    
    # Create the model
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    
    return model

# Define parameters and paths
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
NUM_CLASSES = 5
EPOCHS = 50
BATCH_SIZE = 32

TRAIN_PATH = '/kaggle/input/heart-31/HEART_3_TTV/train' # Replace with your training data path
VAL_PATH = '/kaggle/input/heart-31/HEART_3_TTV/validation' # Replace with your validation data path
TEST_PATH = '/kaggle/input/heart-31/HEART_3_TTV/test' # Replace with your test data path

# Data generators
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    channel_shift_range=0.1
)

val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Flow training images in batches from directory
train_ds = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow validation images in batches from directory
val_ds = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow test images in batches from directory
test_ds = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)  # Input shape for RGB images

# Build the model
model = build_efficientnetb2_model(input_shape, NUM_CLASSES)

# Use ADAMW optimizer
optimizer = optimizers.AdamW(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Add callbacks for early stopping and learning rate reduction
# Add callbacks for early stopping and learning rate reduction
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)

# Train the model with learning rate scheduler and early stopping
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=[early_stopping, reduce_lr])

# Evaluate on validation dataset
val_loss, val_accuracy = model.evaluate(val_ds)

# Evaluate on test dataset
test_loss, test_accuracy = model.evaluate(test_ds)

# Save metrics to Excel files (you can define this function as needed)
# save_metrics_to_excel(history, val_loss, val_accuracy, test_loss, test_accuracy, training_file, validation_file, test_file)

# Print final metrics
print("Training Metrics:")
print("Train Loss:", history.history['loss'][-1])
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Metrics:")
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)
print("Test Metrics:")
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def generate_classification_report_and_confusion_matrix(model, dataset, dataset_type):
    # Predict classes
    predictions = model.predict(dataset)
    y_pred = np.argmax(predictions, axis=1)  # Convert predicted probabilities to class labels
    y_true = dataset.classes

    # Get class labels
    class_labels = list(dataset.class_indices.keys())

    # Generate classification report
    report = classification_report(y_true, y_pred, target_names=class_labels)
    print(f"\n{dataset_type} Classification Report:")
    print(report)

    # Generate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, class_labels, f'Confusion Matrix - {dataset_type}')

    return report

def plot_confusion_matrix(cm, class_labels, title):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=class_labels, yticklabels=class_labels, cbar=False,
                annot_kws={"weight": "bold"})
    plt.title(title, fontweight='bold')
    plt.xlabel('PREDICTED LABEL', fontweight='bold')
    plt.ylabel('TRUE LABEL', fontweight='bold')
    plt.xticks(fontweight='bold')
    plt.yticks(fontweight='bold')
    plt.show()

# Assuming `model`, `train_ds`, `val_ds`, and `test_ds` are already defined elsewhere in your code

# Generate classification reports and confusion matrices
train_report = generate_classification_report_and_confusion_matrix(model, train_ds, 'Training')
val_report = generate_classification_report_and_confusion_matrix(model, val_ds, 'Validation')
test_report = generate_classification_report_and_confusion_matrix(model, test_ds, 'Test')


In [ ]:
model.save('effb2_ADAMW.h5')

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers, regularizers, callbacks
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np

def build_efficientnetb2_model(input_shape, num_classes):
    # Load the pre-trained EfficientNetB2 model
    base_model = EfficientNetB2(include_top=False, weights='imagenet', input_shape=input_shape)
    
    # Set the base model layers to be trainable (no freezing)
    base_model.trainable = True
    
    # Add custom head for classification with increased Dropout and Elastic Net regularization
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dropout(0.4)(x)  # Increase dropout rate to 0.6
    x = layers.BatchNormalization()(x)  # Add batch normalization
    x = layers.Dense(num_classes, 
                     activation='softmax', 
                     kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)  # Elastic Net regularization
    
    # Create the model
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    
    return model

# Define parameters and paths
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
NUM_CLASSES = 5
EPOCHS = 50
BATCH_SIZE = 32

TRAIN_PATH = '/kaggle/input/heart-31/HEART_3_TTV/train'  # Replace with your training data path
VAL_PATH = '/kaggle/input/heart-31/HEART_3_TTV/validation'  # Replace with your validation data path
TEST_PATH = '/kaggle/input/heart-31/HEART_3_TTV/test'  # Replace with your test data path

# Data generators
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    channel_shift_range=0.1
)

val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Flow training images in batches from directory
train_ds = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow validation images in batches from directory
val_ds = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow test images in batches from directory
test_ds = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)  # Input shape for RGB images

# Build the model
model = build_efficientnetb2_model(input_shape, NUM_CLASSES)

# Use Adam optimizer
optimizer = optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Add callbacks for early stopping and learning rate reduction
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)

# Train the model with learning rate scheduler and early stopping
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=[early_stopping, reduce_lr])

# Evaluate on validation dataset
val_loss, val_accuracy = model.evaluate(val_ds)

# Evaluate on test dataset
test_loss, test_accuracy = model.evaluate(test_ds)

# Save metrics to Excel files (you can define this function as needed)
# save_metrics_to_excel(history, val_loss, val_accuracy, test_loss, test_accuracy, training_file, validation_file, test_file)

# Print final metrics
print("Training Metrics:")
print("Train Loss:", history.history['loss'][-1])
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Metrics:")
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)
print("Test Metrics:")
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def generate_classification_report_and_confusion_matrix(model, dataset, dataset_type):
    # Predict classes
    predictions = model.predict(dataset)
    y_pred = np.argmax(predictions, axis=1)  # Convert predicted probabilities to class labels
    y_true = dataset.classes

    # Get class labels
    class_labels = list(dataset.class_indices.keys())

    # Generate classification report
    report = classification_report(y_true, y_pred, target_names=class_labels)
    print(f"\n{dataset_type} Classification Report:")
    print(report)

    # Generate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, class_labels, f'Confusion Matrix - {dataset_type}')

    return report

def plot_confusion_matrix(cm, class_labels, title):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=class_labels, yticklabels=class_labels, cbar=False,
                annot_kws={"weight": "bold"})
    plt.title(title, fontweight='bold')
    plt.xlabel('PREDICTED LABEL', fontweight='bold')
    plt.ylabel('TRUE LABEL', fontweight='bold')
    plt.xticks(fontweight='bold')
    plt.yticks(fontweight='bold')
    plt.show()

# Assuming `model`, `train_ds`, `val_ds`, and `test_ds` are already defined elsewhere in your code

# Generate classification reports and confusion matrices
train_report = generate_classification_report_and_confusion_matrix(model, train_ds, 'Training')
val_report = generate_classification_report_and_confusion_matrix(model, val_ds, 'Validation')
test_report = generate_classification_report_and_confusion_matrix(model, test_ds, 'Test')


In [ ]:
model.save('effb2_721_adam2.h5')

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers, regularizers, callbacks
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np

def build_efficientnetb2_model(input_shape, num_classes):
    # Load the pre-trained EfficientNetB2 model
    base_model = EfficientNetB2(include_top=False, weights='imagenet', input_shape=input_shape)
    
    # Set the base model layers to be trainable (no freezing)
    base_model.trainable = True
    
    # Add custom head for classification with increased Dropout and Elastic Net regularization
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dropout(0.4)(x)  # Increase dropout rate to 0.6
    x = layers.BatchNormalization()(x)  # Add batch normalization
    x = layers.Dense(num_classes, 
                     activation='softmax', 
                     kernel_regularizer=regularizers.l1_l2(l1=0.1, l2=0.1))(x)  # Elastic Net regularization
    
    # Create the model
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    
    return model

# Define parameters and paths
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
NUM_CLASSES = 5
EPOCHS = 50
BATCH_SIZE = 32

TRAIN_PATH = '/kaggle/input/heart-31/HEART_3_TTV/train'  # Replace with your training data path
VAL_PATH = '/kaggle/input/heart-31/HEART_3_TTV/validation'  # Replace with your validation data path
TEST_PATH = '/kaggle/input/heart-31/HEART_3_TTV/test'  # Replace with your test data path

# Data generators
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    channel_shift_range=0.1
)

val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Flow training images in batches from directory
train_ds = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow validation images in batches from directory
val_ds = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow test images in batches from directory
test_ds = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)  # Input shape for RGB images

# Build the model
model = build_efficientnetb2_model(input_shape, NUM_CLASSES)

# Use Adam optimizer
optimizer = optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Add callbacks for early stopping and learning rate reduction
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)

# Train the model with learning rate scheduler and early stopping
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=[early_stopping, reduce_lr])

# Evaluate on validation dataset
val_loss, val_accuracy = model.evaluate(val_ds)

# Evaluate on test dataset
test_loss, test_accuracy = model.evaluate(test_ds)

# Save metrics to Excel files (you can define this function as needed)
# save_metrics_to_excel(history, val_loss, val_accuracy, test_loss, test_accuracy, training_file, validation_file, test_file)

# Print final metrics
print("Training Metrics:")
print("Train Loss:", history.history['loss'][-1])
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Metrics:")
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)
print("Test Metrics:")
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def generate_classification_report_and_confusion_matrix(model, dataset, dataset_type):
    # Predict classes
    predictions = model.predict(dataset)
    y_pred = np.argmax(predictions, axis=1)  # Convert predicted probabilities to class labels
    y_true = dataset.classes

    # Get class labels
    class_labels = list(dataset.class_indices.keys())

    # Generate classification report
    report = classification_report(y_true, y_pred, target_names=class_labels)
    print(f"\n{dataset_type} Classification Report:")
    print(report)

    # Generate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, class_labels, f'Confusion Matrix - {dataset_type}')

    return report

def plot_confusion_matrix(cm, class_labels, title):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=class_labels, yticklabels=class_labels, cbar=False,
                annot_kws={"weight": "bold"})
    plt.title(title, fontweight='bold')
    plt.xlabel('PREDICTED LABEL', fontweight='bold')
    plt.ylabel('TRUE LABEL', fontweight='bold')
    plt.xticks(fontweight='bold')
    plt.yticks(fontweight='bold')
    plt.show()

# Assuming `model`, `train_ds`, `val_ds`, and `test_ds` are already defined elsewhere in your code

# Generate classification reports and confusion matrices
train_report = generate_classification_report_and_confusion_matrix(model, train_ds, 'Training')
val_report = generate_classification_report_and_confusion_matrix(model, val_ds, 'Validation')
test_report = generate_classification_report_and_confusion_matrix(model, test_ds, 'Test')


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers, regularizers, callbacks
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np

def build_efficientnetb2_model(input_shape, num_classes):
    # Load the pre-trained EfficientNetB2 model
    base_model = EfficientNetB2(include_top=False, weights='imagenet', input_shape=input_shape)
    
    # Set the base model layers to be trainable (no freezing)
    base_model.trainable = True
    
    # Add custom head for classification with increased Dropout and Elastic Net regularization
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dropout(0.4)(x)  # Increase dropout rate to 0.6
    x = layers.BatchNormalization()(x)  # Add batch normalization
    x = layers.Dense(num_classes, 
                     activation='softmax', 
                     kernel_regularizer=regularizers.l1_l2(l1=0.1, l2=0.1))(x)  # Elastic Net regularization
    
    # Create the model
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    
    return model

# Define parameters and paths
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
NUM_CLASSES = 5
EPOCHS = 50
BATCH_SIZE = 32

TRAIN_PATH = '/kaggle/input/heart-31/HEART_3_TTV/train'  # Replace with your training data path
VAL_PATH = '/kaggle/input/heart-31/HEART_3_TTV/validation'  # Replace with your validation data path
TEST_PATH = '/kaggle/input/heart-31/HEART_3_TTV/test'  # Replace with your test data path

# Data generators
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    channel_shift_range=0.1
)

val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Flow training images in batches from directory
train_ds = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow validation images in batches from directory
val_ds = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow test images in batches from directory
test_ds = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)  # Input shape for RGB images

# Build the model
model = build_efficientnetb2_model(input_shape, NUM_CLASSES)

# Use Adam optimizer
optimizer = optimizers.Adam(learning_rate=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Add callbacks for early stopping and learning rate reduction
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)

# Train the model with learning rate scheduler and early stopping
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=[early_stopping, reduce_lr])

# Evaluate on validation dataset
val_loss, val_accuracy = model.evaluate(val_ds)

# Evaluate on test dataset
test_loss, test_accuracy = model.evaluate(test_ds)

# Save metrics to Excel files (you can define this function as needed)
# save_metrics_to_excel(history, val_loss, val_accuracy, test_loss, test_accuracy, training_file, validation_file, test_file)

# Print final metrics
print("Training Metrics:")
print("Train Loss:", history.history['loss'][-1])
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Metrics:")
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)
print("Test Metrics:")
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers, regularizers, callbacks
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np

def build_efficientnetb2_model(input_shape, num_classes):
    # Load the pre-trained EfficientNetB2 model
    base_model = EfficientNetB2(include_top=False, weights='imagenet', input_shape=input_shape)
    
    # Set the base model layers to be trainable (no freezing)
    base_model.trainable = True
    
    # Add custom head for classification with increased Dropout and Elastic Net regularization
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dropout(0.4)(x)  # Increase dropout rate to 0.4
    x = layers.BatchNormalization()(x)  # Add batch normalization
    x = layers.Dense(num_classes, 
                     activation='softmax', 
                     kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)  # Elastic Net regularization
    
    # Create the model
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    
    return model

# Define parameters and paths
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
NUM_CLASSES = 5
EPOCHS = 50
BATCH_SIZE = 32

TRAIN_PATH = '/kaggle/input/heart-31/HEART_3_TTV/train'  # Replace with your training data path
VAL_PATH = '/kaggle/input/heart-31/HEART_3_TTV/validation'  # Replace with your validation data path
TEST_PATH = '/kaggle/input/heart-31/HEART_3_TTV/test'  # Replace with your test data path

# Data generators
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    channel_shift_range=0.1
)

val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Flow training images in batches from directory
train_ds = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow validation images in batches from directory
val_ds = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow test images in batches from directory
test_ds = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)  # Input shape for RGB images

# Build the model
model = build_efficientnetb2_model(input_shape, NUM_CLASSES)

# Use SGD optimizer with momentum
optimizer = optimizers.SGD(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Add callbacks for early stopping and learning rate reduction
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)

# Train the model with learning rate scheduler and early stopping
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=[early_stopping, reduce_lr])

# Evaluate on validation dataset
val_loss, val_accuracy = model.evaluate(val_ds)

# Evaluate on test dataset
test_loss, test_accuracy = model.evaluate(test_ds)

# Save metrics to Excel files (you can define this function as needed)
# save_metrics_to_excel(history, val_loss, val_accuracy, test_loss, test_accuracy, training_file, validation_file, test_file)

# Print final metrics
print("Training Metrics:")
print("Train Loss:", history.history['loss'][-1])
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Metrics:")
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)
print("Test Metrics:")
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def generate_classification_report_and_confusion_matrix(model, dataset, dataset_type):
    # Predict classes
    predictions = model.predict(dataset)
    y_pred = np.argmax(predictions, axis=1)  # Convert predicted probabilities to class labels
    y_true = dataset.classes

    # Get class labels
    class_labels = list(dataset.class_indices.keys())

    # Generate classification report
    report = classification_report(y_true, y_pred, target_names=class_labels)
    print(f"\n{dataset_type} Classification Report:")
    print(report)

    # Generate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, class_labels, f'Confusion Matrix - {dataset_type}')

    return report

def plot_confusion_matrix(cm, class_labels, title):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=class_labels, yticklabels=class_labels, cbar=False,
                annot_kws={"weight": "bold"})
    plt.title(title, fontweight='bold')
    plt.xlabel('PREDICTED LABEL', fontweight='bold')
    plt.ylabel('TRUE LABEL', fontweight='bold')
    plt.xticks(fontweight='bold')
    plt.yticks(fontweight='bold')
    plt.show()

# Assuming `model`, `train_ds`, `val_ds`, and `test_ds` are already defined elsewhere in your code

# Generate classification reports and confusion matrices
train_report = generate_classification_report_and_confusion_matrix(model, train_ds, 'Training')
val_report = generate_classification_report_and_confusion_matrix(model, val_ds, 'Validation')
test_report = generate_classification_report_and_confusion_matrix(model, test_ds, 'Test')


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers, losses, regularizers, callbacks
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def build_efficientnetb2_model(input_shape, num_classes):
    # Load the pre-trained EfficientNetB2 model
    base_model = EfficientNetB2(include_top=False, weights='imagenet', input_shape=input_shape)
    
    # Set the base model layers to be trainable (no freezing)
    base_model.trainable = True
    
    # Add custom head for classification with Dropout and L1, L2 regularization
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dropout(0.5)(x)  # Add dropout layer
    x = layers.BatchNormalization()(x)  # Add batch normalization
    x = layers.Dense(num_classes, 
                     activation='softmax', 
                     kernel_regularizer=regularizers.l2(0.01),  # L2 regularization
                     activity_regularizer=regularizers.l1(0.01))(x)  # L1 regularization
    
    # Create the model
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    
    return model

# Define parameters and paths
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
NUM_CLASSES = 5
EPOCHS = 50
BATCH_SIZE = 32

TRAIN_PATH = '/kaggle/input/heart-31/HEART_3_TTV/train' # Replace with your training data path
VAL_PATH = '/kaggle/input/heart-31/HEART_3_TTV/validation' # Replace with your validation data path
TEST_PATH = '/kaggle/input/heart-31/HEART_3_TTV/test' # Replace with your test data path

# Data generators
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    channel_shift_range=0.1
)

val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Flow training images in batches from directory
train_ds = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow validation images in batches from directory
val_ds = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow test images in batches from directory
test_ds = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)  # Input shape for RGB images

# Build the model
model = build_efficientnetb2_model(input_shape, NUM_CLASSES)

# Use Adam optimizer
optimizer = optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Add callbacks for early stopping and learning rate reduction
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)

# Train the model with learning rate scheduler and early stopping
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=[early_stopping, reduce_lr])

# Evaluate on validation dataset
val_loss, val_accuracy = model.evaluate(val_ds)

# Evaluate on test dataset
test_loss, test_accuracy = model.evaluate(test_ds)

# Save metrics to Excel files (you can define this function as needed)
# save_metrics_to_excel(history, val_loss, val_accuracy, test_loss, test_accuracy, training_file, validation_file, test_file)

# Print final metrics
print("Training Metrics:")
print("Train Loss:", history.history['loss'][-1])
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Metrics:")
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)
print("Test Metrics:")
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers, regularizers, callbacks
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np

def build_efficientnetb2_model(input_shape, num_classes):
    # Load the pre-trained EfficientNetB2 model
    base_model = EfficientNetB2(include_top=False, weights='imagenet', input_shape=input_shape)
    
    # Set the base model layers to be trainable (no freezing)
    base_model.trainable = True
    
    # Add custom head for classification with increased Dropout and Elastic Net regularization
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dropout(0.4)(x)  # Increase dropout rate to 0.4
    x = layers.BatchNormalization()(x)  # Add batch normalization
    x = layers.Dense(num_classes, 
                     activation='softmax', 
                     kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)  # Elastic Net regularization
    
    # Create the model
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    
    return model

# Define parameters and paths
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
NUM_CLASSES = 5
EPOCHS = 70
BATCH_SIZE = 32

TRAIN_PATH = '/kaggle/input/heart-31/HEART_3_TTV/train'  # Replace with your training data path
VAL_PATH = '/kaggle/input/heart-31/HEART_3_TTV/validation'  # Replace with your validation data path
TEST_PATH = '/kaggle/input/heart-31/HEART_3_TTV/test'  # Replace with your test data path

# Data generators
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    channel_shift_range=0.1
)

val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Flow training images in batches from directory
train_ds = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow validation images in batches from directory
val_ds = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow test images in batches from directory
test_ds = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)  # Input shape for RGB images

# Build the model
model = build_efficientnetb2_model(input_shape, NUM_CLASSES)

# Use SGD optimizer with momentum
optimizer = optimizers.SGD(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Add callbacks for early stopping and learning rate reduction
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)

# Train the model with learning rate scheduler and early stopping
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=[early_stopping, reduce_lr])

# Evaluate on validation dataset
val_loss, val_accuracy = model.evaluate(val_ds)

# Evaluate on test dataset
test_loss, test_accuracy = model.evaluate(test_ds)

# Save metrics to Excel files (you can define this function as needed)
# save_metrics_to_excel(history, val_loss, val_accuracy, test_loss, test_accuracy, training_file, validation_file, test_file)

# Print final metrics
print("Training Metrics:")
print("Train Loss:", history.history['loss'][-1])
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Metrics:")
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)
print("Test Metrics:")
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def generate_classification_report_and_confusion_matrix(model, dataset, dataset_type):
    # Predict classes
    predictions = model.predict(dataset)
    y_pred = np.argmax(predictions, axis=1)  # Convert predicted probabilities to class labels
    y_true = dataset.classes

    # Get class labels
    class_labels = list(dataset.class_indices.keys())

    # Generate classification report
    report = classification_report(y_true, y_pred, target_names=class_labels)
    print(f"\n{dataset_type} Classification Report:")
    print(report)

    # Generate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, class_labels, f'Confusion Matrix - {dataset_type}')

    return report

def plot_confusion_matrix(cm, class_labels, title):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=class_labels, yticklabels=class_labels, cbar=False,
                annot_kws={"weight": "bold"})
    plt.title(title, fontweight='bold')
    plt.xlabel('PREDICTED LABEL', fontweight='bold')
    plt.ylabel('TRUE LABEL', fontweight='bold')
    plt.xticks(fontweight='bold')
    plt.yticks(fontweight='bold')
    plt.show()

# Assuming `model`, `train_ds`, `val_ds`, and `test_ds` are already defined elsewhere in your code

# Generate classification reports and confusion matrices
train_report = generate_classification_report_and_confusion_matrix(model, train_ds, 'Training')
val_report = generate_classification_report_and_confusion_matrix(model, val_ds, 'Validation')
test_report = generate_classification_report_and_confusion_matrix(model, test_ds, 'Test')


In [ ]:
model.save('efficientnetb2_721_sgd_70.h5')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers, losses, callbacks

def extract_features(model, generator, num_samples, feature_shape):
    # Create an empty array to hold the features
    features = np.zeros(shape=(num_samples, np.prod(feature_shape)))
    labels = np.zeros(shape=(num_samples, generator.num_classes))
    
    i = 0
    for inputs_batch, labels_batch in generator:
        batch_size = inputs_batch.shape[0]
        features_batch = model.predict(inputs_batch)
        features[i * batch_size: i * batch_size + batch_size] = features_batch.reshape(batch_size, -1)
        labels[i * batch_size: i * batch_size + batch_size] = labels_batch
        i += 1
        if i * batch_size + batch_size >= num_samples:
            break
    
    return features, labels

# Define parameters and paths
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
NUM_CLASSES = 5
BATCH_SIZE = 32
EPOCHS = 50

TRAIN_PATH = '/kaggle/input/heart-31/HEART_3_TTV/train'  # Replace with your training data path
VAL_PATH = '/kaggle/input/heart-31/HEART_3_TTV/validation'  # Replace with your validation data path
TEST_PATH = '/kaggle/input/heart-31/HEART_3_TTV/test'  # Replace with your test data path

# Data generators
datagen = ImageDataGenerator()

# Flow training images in batches from directory
train_ds = datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Flow validation images in batches from directory
val_ds = datagen.flow_from_directory(
    VAL_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Flow test images in batches from directory
test_ds = datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Load the pre-trained EfficientNetB2 model
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)
base_model = EfficientNetB2(include_top=False, weights='imagenet', input_shape=input_shape)
base_model.trainable = False  # Freeze the base model

# Calculate the feature shape
feature_shape = base_model.output_shape[1:]

# Extract features
train_features, train_labels = extract_features(base_model, train_ds, train_ds.samples, feature_shape)
val_features, val_labels = extract_features(base_model, val_ds, val_ds.samples, feature_shape)
test_features, test_labels = extract_features(base_model, test_ds, test_ds.samples, feature_shape)

# Flatten the labels
train_labels = np.argmax(train_labels, axis=1)
val_labels = np.argmax(val_labels, axis=1)
test_labels = np.argmax(test_labels, axis=1)

# Train XGBoost model with a specified learning rate
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=NUM_CLASSES, learning_rate=0.01)
xgb_model.fit(train_features, train_labels, eval_set=[(val_features, val_labels)], early_stopping_rounds=10, verbose=True)

# Evaluate on training dataset
train_predictions = xgb_model.predict(train_features)
train_accuracy = np.mean(train_predictions == train_labels)

# Evaluate on validation dataset
val_predictions = xgb_model.predict(val_features)
val_accuracy = np.mean(val_predictions == val_labels)

# Evaluate on test dataset
test_predictions = xgb_model.predict(test_features)
test_accuracy = np.mean(test_predictions == test_labels)

# Print final metrics
print("Training Metrics:")
print("Train Accuracy:", train_accuracy)
print("Validation Metrics:")
print("Validation Accuracy:", val_accuracy)
print("Test Metrics:")
print("Test Accuracy:", test_accuracy)


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers, regularizers, callbacks
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

def build_efficientnetb3_model(input_shape, num_classes):
    # Load the pre-trained EfficientNetB3 model
    base_model = EfficientNetB3(include_top=False, weights='imagenet', input_shape=input_shape)
    
    # Set the base model layers to be trainable (no freezing)
    base_model.trainable = True
    
    # Add custom head for classification with increased Dropout and Elastic Net regularization
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dropout(0.4)(x)  # Increase dropout rate to 0.4
    x = layers.BatchNormalization()(x)  # Add batch normalization
    x = layers.Dense(num_classes, 
                     activation='softmax', 
                     kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)  # Elastic Net regularization
    
    # Create the model
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    
    return model

# Define parameters and paths
IMAGE_WIDTH = 300
IMAGE_HEIGHT = 300
NUM_CLASSES = 5
EPOCHS = 50
BATCH_SIZE = 32


TRAIN_PATH = '/kaggle/input/heart-31/HEART_3_TTV/train'  # Replace with your training data path
VAL_PATH = '/kaggle/input/heart-31/HEART_3_TTV/validation'  # Replace with your validation data path
TEST_PATH = '/kaggle/input/heart-31/HEART_3_TTV/test'  


# Data generators
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    channel_shift_range=0.1
)

val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Flow training images in batches from directory
train_ds = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow validation images in batches from directory
val_ds = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow test images in batches from directory
test_ds = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)  # Input shape for RGB images

# Build the model
model = build_efficientnetb3_model(input_shape, NUM_CLASSES)

# Use Adam optimizer
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Add callbacks for early stopping and learning rate reduction
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)

# Train the model with learning rate scheduler and early stopping
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=[early_stopping, reduce_lr])

# Evaluate on validation dataset
val_loss, val_accuracy = model.evaluate(val_ds)

# Evaluate on test dataset
test_loss, test_accuracy = model.evaluate(test_ds)

# Save metrics to Excel files (you can define this function as needed)
# save_metrics_to_excel(history, val_loss, val_accuracy, test_loss, test_accuracy, training_file, validation_file, test_file)

# Print final metrics
print("Training Metrics:")
print("Train Loss:", history.history['loss'][-1])
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Metrics:")
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)
print("Test Metrics:")
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def generate_classification_report_and_confusion_matrix(model, dataset, dataset_type):
    # Predict classes
    predictions = model.predict(dataset)
    y_pred = np.argmax(predictions, axis=1)  # Convert predicted probabilities to class labels
    y_true = dataset.classes

    # Get class labels
    class_labels = list(dataset.class_indices.keys())

    # Generate classification report
    report = classification_report(y_true, y_pred, target_names=class_labels)
    print(f"\n{dataset_type} Classification Report:")
    print(report)

    # Generate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, class_labels, f'Confusion Matrix - {dataset_type}')

    return report

def plot_confusion_matrix(cm, class_labels, title):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=class_labels, yticklabels=class_labels, cbar=False,
                annot_kws={"weight": "bold"})
    plt.title(title, fontweight='bold')
    plt.xlabel('PREDICTED LABEL', fontweight='bold')
    plt.ylabel('TRUE LABEL', fontweight='bold')
    plt.xticks(fontweight='bold')
    plt.yticks(fontweight='bold')
    plt.show()

# Assuming `model`, `train_ds`, `val_ds`, and `test_ds` are already defined elsewhere in your code

# Generate classification reports and confusion matrices
train_report = generate_classification_report_and_confusion_matrix(model, train_ds, 'Training')
val_report = generate_classification_report_and_confusion_matrix(model, val_ds, 'Validation')
test_report = generate_classification_report_and_confusion_matrix(model, test_ds, 'Test')


In [ ]:
model.save('effB3_adam.h5')

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers, regularizers, callbacks
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

def build_efficientnetb3_model(input_shape, num_classes):
    # Load the pre-trained EfficientNetB3 model
    base_model = EfficientNetB3(include_top=False, weights='imagenet', input_shape=input_shape)
    
    # Set the base model layers to be trainable (no freezing)
    base_model.trainable = True
    
    # Add custom head for classification with increased Dropout and Elastic Net regularization
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dropout(0.4)(x)  # Increase dropout rate to 0.4
    x = layers.BatchNormalization()(x)  # Add batch normalization
    x = layers.Dense(num_classes, 
                     activation='softmax', 
                     kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)  # Elastic Net regularization
    
    # Create the model
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    
    return model

# Define parameters and paths
IMAGE_WIDTH = 300
IMAGE_HEIGHT = 300
NUM_CLASSES = 5
EPOCHS = 50
BATCH_SIZE = 32


TRAIN_PATH = '/kaggle/input/heart-31/HEART_3_TTV/train'  # Replace with your training data path
VAL_PATH = '/kaggle/input/heart-31/HEART_3_TTV/validation'  # Replace with your validation data path
TEST_PATH = '/kaggle/input/heart-31/HEART_3_TTV/test'  


# Data generators
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    channel_shift_range=0.1
)

val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Flow training images in batches from directory
train_ds = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow validation images in batches from directory
val_ds = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow test images in batches from directory
test_ds = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)  # Input shape for RGB images

# Build the model
model = build_efficientnetb3_model(input_shape, NUM_CLASSES)

# Use RMSprop optimizer
optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Add callbacks for early stopping and learning rate reduction
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)

# Train the model with learning rate scheduler and early stopping
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=[early_stopping, reduce_lr])

# Evaluate on validation dataset
val_loss, val_accuracy = model.evaluate(val_ds)

# Evaluate on test dataset
test_loss, test_accuracy = model.evaluate(test_ds)

# Save metrics to Excel files (you can define this function as needed)
# save_metrics_to_excel(history, val_loss, val_accuracy, test_loss, test_accuracy, training_file, validation_file, test_file)

# Print final metrics
print("Training Metrics:")
print("Train Loss:", history.history['loss'][-1])
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Metrics:")
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)
print("Test Metrics:")
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
